In [18]:
pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install langchain

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.3
    Uninstalling numpy-2.3.3:
      Successfully uninstalled numpy-2.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.16.2 which is incompatible.
tensorflow 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.20.0 which is incompatible.
fuzzy-c-means 1.7.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.19.2 which is incompatible.
en-core-sci-md 0.5.0 requires spacy<3.3.0,>=3.2.3, but you have spacy 3.8.4 which is incompatible.
N

In [20]:
pip install langchain_huggingface

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install chromadb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [22]:
# !pip install "opentelemetry-api<1.37" "opentelemetry-sdk<1.37" \
#             "opentelemetry-exporter-otlp-proto-grpc<1.37" --force-reinstall


# This notebook contains Step 2 : Answer Generation and  Step 3 Assertions part of the pipeline

In [1]:
import json
import os
import numpy as np
import pandas as pd
import json
import re
import pandas as pd
from tqdm import tqdm  
from sentence_transformers import CrossEncoder
from transformers import pipeline

2025-09-27 15:56:21.381782: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-27 15:56:21.397377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759006581.413054  868651 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759006581.418297  868651 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-27 15:56:21.437506: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import os
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import warnings
from transformers.utils import logging as hf_logging
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer, util
import pandas as pd

In [3]:
import warnings

os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [4]:
hf_logging.set_verbosity_error()
warnings.filterwarnings("ignore", category=FutureWarning)

### Fetch data from the ChromaDb Store

In [24]:
#Initializing cross encoder

cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [27]:
def cosine_similarity(prediction, ground_truth, embedder):
    emb_pred = embedder.encode(prediction, convert_to_tensor=True)
    emb_gt   = embedder.encode(ground_truth, convert_to_tensor=True)
    return util.cos_sim(emb_pred, emb_gt).item() 

In [28]:
def retrieve_chunks(chroma_collection, question, id=None, k=10):
    where_clause = {"question_id": str(id)} if id else None

    result = chroma_collection.query(
        query_texts=[question],
        n_results=k,
        where=where_clause
    )

    pairs = [(question, doc) for doc in result["documents"][0]]
    scores = cross_encoder.predict(pairs)
    
    # sort by score descending
    reranked = [doc for _, doc in sorted(zip(scores, result["documents"][0]), key=lambda x: x[0], reverse=True)]
    final_chunks = reranked[:3]

    context_text = " ".join(final_chunks)

    
    context_text = " ".join(result["documents"][0])  # combine list of strings into one
    clean_chunk = re.sub(r"\$.*?\$", "", context_text)  
 
    # Return list of text chunks
    return clean_chunk


In [33]:
def qa_with_confidence(qa_model, qa_tokenizer, question, context):
    inputs = qa_tokenizer(
        question,
        context,
        return_tensors="pt",
        max_length=512,
        truncation=True
    )
    with torch.no_grad():
        outputs = qa_model(**inputs)

    start_idx = outputs.start_logits.argmax()
    end_idx   = outputs.end_logits.argmax()

    answer_tokens = inputs.input_ids[0, start_idx:end_idx+1]
    decoded_answer = qa_tokenizer.decode(answer_tokens, skip_special_tokens=True)

    start_probs = F.softmax(outputs.start_logits, dim=-1)
    end_probs   = F.softmax(outputs.end_logits, dim=-1)
    confidence  = (start_probs[0, start_idx] * end_probs[0, end_idx]).item()

    return decoded_answer, confidence


In [34]:
def run_answer_extraction_and_evaluation(qa_model, qa_tokenizer, chroma_collection, questions, embedder):
    results = []
    for _, row in tqdm(questions.iterrows()):
        chunks = retrieve_chunks(chroma_collection, row.question, row.id)
        
        if not chunks or chunks.strip() == "":
            print(f"Skipping {row.id} (no context retrieved)")
            continue

        #Evaluation
        pred, conf = qa_with_confidence(qa_model, qa_tokenizer, row.question, chunks)
        cos        = cosine_similarity(pred, str(row.answer), embedder)
        
        results.append({
            "question_id": row.id,
            "question" : row.question,
            "model_answer": pred,
            "ground_truth": str(row.answer),
            "confidence": conf,
            "cosine_sim": cos
        })
    
    return pd.DataFrame(results)

In [37]:
def run_pipeline(db_path: str, base_configs: dict, embedding_models: list, generation_models: list):
    client = chromadb.PersistentClient(path=db_path)

    for ds_name, ds_info in base_configs.items():
        for emb_model in embedding_models:
            for gen_model in generation_models:

                collection_name = f"{ds_info['collection']}_{emb_model}"

                print(f"  Running pipeline for {ds_name}")
                print(f"   Embedding Model : {emb_model}")
                print(f"   Generation Model: {gen_model}")
                print(f"   Collection Name : {collection_name}")

                # Connect to the collection with the correct embedding function
                embedding_fn = SentenceTransformerEmbeddingFunction(model_name=emb_model)
                chroma_collection = client.get_collection(
                    name=collection_name,
                    embedding_function=embedding_fn
                )

                # Load questions
                questions_df = pd.read_csv(ds_info["questions"])

                # Load QA model
                qa_tokenizer = AutoTokenizer.from_pretrained(gen_model)
                qa_model     = AutoModelForQuestionAnswering.from_pretrained(gen_model)

                embedder = SentenceTransformer(emb_model) 

                # Run QA + evaluation 
                scores_df = run_answer_extraction_and_evaluation(
                    qa_model, qa_tokenizer, chroma_collection, questions_df, embedder
                )

                safe_emb = emb_model.replace("/", "-")      # e.g. all-MiniLM-L6-v2
                safe_gen = gen_model.replace("/", "-")      # e.g. deepset-roberta-base-squad2
                file_name = f"scores_{ds_name}_{safe_emb}_{safe_gen}.csv"
                
                # Save results
                scores_df.to_csv(file_name, index=False)
                print(f"   Saved results to: {file_name}")

                print(f"   Average Confidence : {scores_df['confidence'].mean():.4f}")
                print(f"   Average Cosine Sim : {scores_df['cosine_sim'].mean():.4f}")


In [ ]:
# All embedding models 
EMBEDDING_MODELS = [
    "all-MiniLM-L6-v2",
    "all-mpnet-base-v2",
    "all-MiniLM-L12-v2"
]

# All generation (QA) models
GENERATION_MODELS = [
    "deepset/tinyroberta-squad2",
    "deepset/roberta-base-squad2",
    "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"
]

BASE_CONFIGS = {
    "HotpotQA": {
        "collection": "hpqa_data_collection",  
        "questions": "processed_hotpot_df.csv"
    },
    "Qasper": {
        "collection": "qasper_data_collection",
        "questions": "processed_qasper_data.csv"
    }
}

DATASET_PATH = "./ChromaDb"
run_pipeline(DATASET_PATH, BASE_CONFIGS, EMBEDDING_MODELS, GENERATION_MODELS)


  Running pipeline for HotpotQA
   Embedding Model : all-MiniLM-L6-v2
   Generation Model: deepset/tinyroberta-squad2
   Collection Name : hpqa_data_collection_all-MiniLM-L6-v2


7405it [17:38,  6.99it/s]


   Saved results to: scores_HotpotQA_all-MiniLM-L6-v2_deepset-tinyroberta-squad2.csv
   Average Confidence : 0.7682
   Average Cosine Sim : 0.2997
  Running pipeline for HotpotQA
   Embedding Model : all-MiniLM-L6-v2
   Generation Model: deepset/roberta-base-squad2
   Collection Name : hpqa_data_collection_all-MiniLM-L6-v2


3589it [12:42,  4.93it/s]